In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

In [2]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode="categorical")
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [5]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)
    

In [6]:
model = model_maker()
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.legacy.Adam(0.001),
              metrics=["acc"])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
8/8 [==============================] - 6s 756ms/step - loss: 0.6837 - acc: 0.7060 - val_loss: 0.1652 - val_acc: 0.9480
Epoch 2/10
8/8 [==============================] - 6s 721ms/step - loss: 0.2278 - acc: 0.8980 - val_loss: 0.1243 - val_acc: 0.9540
Epoch 3/10
8/8 [==============================] - 6s 716ms/step - loss: 0.1359 - acc: 0.9480 - val_loss: 0.0991 - val_acc: 0.9640
Epoch 4/10
8/8 [==============================] - 6s 724ms/step - loss: 0.1380 - acc: 0.9420 - val_loss: 0.1023 - val_acc: 0.9540
Epoch 5/10
8/8 [==============================] - 6s 730ms/step - loss: 0.0954 - acc: 0.9680 - val_loss: 0.0932 - val_acc: 0.9680
Epoch 6/10
8/8 [==============================] - 6s 733ms/step - loss: 0.0724 - acc: 0.9780 - val_loss: 0.0886 - val_acc: 0.9700
Epoch 7/10
8/8 [==============================] - 6s 718ms/step - loss: 0.0768 - acc: 0.9680 - val_loss: 0.0870 - val_acc: 0.9680
Epoch 8/10
8/8 [==============================] - 6s 722ms/step - loss: 0.0726 - acc: 0.97

In [7]:
model.save('model.h5')

/Users/oratta/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model("model.h5")

In [9]:
img_path = "../sample-images/dog.jpg"
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

1/1 [==============================] - 0s 164ms/step
[[0.12561621 0.8743838 ]]
{'cat': 0, 'dog': 1}


In [12]:
##結果の分析##
# 最も確率が高いとされた画像はどれ？
# 最も確率が低いとされた画像はどれ？
# 確率が高いのに、予測が誤っていた画像はどれ？

IMG_WIDTH, IMG_HEIGHT = 224,224
VALIDATION_DATA_DIR = "data/val"
VALIDATION_BATCH_SIZE = 64

# データジェネレータ
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=VALIDATION_BATCH_SIZE,
    shuffle=False,
    class_mode="categorical")
ground_truth = validation_generator.classes

Found 500 images belonging to 2 classes.


In [13]:
predictions = model.predict(validation_generator)

8/8 [==============================] - 2s 231ms/step


In [16]:
# 分析のために、画像にインデックスと予測結果と正解flagを持たせたディクショナリの作成
prediction_table = {}
for index, val in enumerate(predictions):
    index_of_highest_probability = np.argmax(val) #一番確率が高かった
    value_of_highest_probability = val[index_of_highest_probability]
    prediction_table[index] = [value_of_highest_probability, index_of_highest_probability, ground_truth[index]]
assert len(predictions) == len(ground_truth) == len(prediction_table)

In [17]:
### 途中までやったけど、分析は別のファイルでやってみる